# LFP example 


In [240]:

from bayes_window.generative_models import generate_fake_lfp
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import visualization


## Make fake data

In [273]:

df, df_monster, index_cols, _ = make_fake_lfp()

In [275]:
reload(visualization)
visualization.plot_data(df=df,x='stim',y='Power',color='mouse')

alt.LayerChart(...)

In [279]:
c1=visualization.plot_data(df=df,x='stim',y='Log power').properties(width=60)
c1.facet('mouse')

alt.FacetChart(...)

In [280]:
reload(visualization)
visualization.plot_data_slope_trials(df=df,x='stim',y='Power',detail='i_trial').properties().facet(column='mouse')


alt.FacetChart(...)

In [281]:
reload(visualization)
c2=visualization.plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

## Fit and plot

In [300]:
from bayes_window import workflow
from bayes_window import models
from bayes_window import visualization
from importlib import reload
reload(workflow)
reload(visualization)
reload(models)

bw=workflow.BayesWindow(df,y='Log power', levels=('stim', 'mouse'))
bw.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
              do_make_change='divide', dist_y='normal')

n(Divergences) = 0


KeyError: "None of ['mouse'] are in the columns"

In [ ]:
reload(visualization)
visualization.plot_posterior(df=bw.data_and_posterior, title=f'Log power', )


In [ ]:
bw.plot_posteriors_slopes(add_box=False, independent_axes=True,)

In [73]:
posterior_mean = (trace[b_name].mean(['chain', 'draw']).values)

np.concatenate((hpd, posterior_mean), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [74]:
np.hstack((hpd.values, posterior_mean))

array([0.49179018, 3.98950648, 2.26417184])

In [55]:
posterior_mean

array([[2.2641718],
       [2.2641718]], dtype=float32)

In [105]:
hpd_condition.ndim

1

In [173]:
import arviz as az
import numpy as np,xarray as xr,pandas as pd
#az.hdi(bw.trace.b_stim_per_condition).b_stim_per_condition
trace=bw.trace
b_name='b_stim_per_condition'
hpd = az.hdi(trace)[b_name]
mean=xr.DataArray([trace[b_name].mean(['chain', 'draw']).values
                  ],coords={'hdi':["mean"],},dims='hdi')
xar=xr.concat([hpd,mean],'hdi')
df=xar.to_dataframe()#.reset_index()


df=df.pivot_table(columns='hdi').reset_index(drop=True)
df#.drop('hdi',axis=1)#

RangeIndex(start=0, stop=1, step=1)

In [ ]:
try:
    posterior_mean = trace[b_name].mean(['chain', 'draw']).values[:, None]
except IndexError:
    posterior_mean = (trace[b_name].mean(['chain', 'draw']).values)

hpd_condition = xr.DataArray(np.hstack((hpd, posterior_mean)),
                             dims=[ 'CI'],
                             coords=[ ['ci_start', 'ci_end', 'mean']])
hpd_condition

In [126]:
import arviz as az
import numpy as np,xarray as xr
#az.hdi(bw.trace.b_stim_per_condition).b_stim_per_condition
trace=bw.trace
b_name='b_stim_per_condition'
hpd = az.hdi(trace)[b_name]
print(hpd)
try:
    posterior_mean = trace[b_name].mean(['chain', 'draw']).values[:, None]
except IndexError:
    posterior_mean = (trace[b_name].mean(['chain', 'draw']).values)

hpd_condition = xr.DataArray(np.hstack((hpd, posterior_mean)),
                             dims=[ 'CI'],
                             coords=[ ['ci_start', 'ci_end', 'mean']])
hpd_condition.to_dataframe('cx')


<xarray.DataArray 'b_stim_per_condition' (hdi: 2)>
array([0.49179018, 3.98950648])
Coordinates:
  * hdi      (hdi) <U6 'lower' 'higher'


,cx
CI,
ci_start,0.491790
ci_end,3.989506
mean,2.264172


# Evaluate fit

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

Plot of a ROC curve for a specific class

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

